In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
train.head()

In [ ]:
test=pd.read_csv("../input/digit-recognizer/test.csv")
test.head()

In [ ]:
import keras
from sklearn.model_selection import train_test_split 
def preprocessing(train, test, split_train_size = 0.1):

    xr= train.drop(["label"],axis = 1)
    yr= train["label"]
    xr=xr/ 255.0
    test=test/ 255.0
    xr=xr.values.reshape(-1,28,28,1)
    xval=test.values.reshape(-1,28,28,1)
    yr= keras.utils.to_categorical(yr, num_classes = 10)
    xr,xt,yr,yt= train_test_split(xr,yr, test_size = split_train_size, random_state=42)
    return xr,xt,yr,yt,xval
xr,xt,yr,yt,xval= preprocessing(train, test)

In [ ]:
xr.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Flatten,MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.layers import Activation, Convolution2D, Dropout, Conv2D,AveragePooling2D, BatchNormalization,Flatten,GlobalAveragePooling2D
from keras import layers
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau

In [ ]:
l2_reg=0.001
opt=Adam(lr=0.001)

In [ ]:
l2_reg=0.001
opt=Adam(lr=0.001)
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(2,2), input_shape=(28,28,1), activation='relu',kernel_regularizer=l2(l2_reg)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.1))
model.add(Conv2D(filters=64, kernel_size=(2,2), activation='relu',kernel_regularizer=l2(l2_reg)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.1))
model.add(Conv2D(filters=128, kernel_size=(2,2), activation='relu',kernel_regularizer=l2(l2_reg)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history=model.fit(xr,yr,batch_size=128,epochs=20,verbose=1,validation_split=0.33)

In [ ]:
import matplotlib.pyplot as plt
figure=plt.figure(figsize=(15,15))
ax=figure.add_subplot(121)
ax.plot(history.history['accuracy'])
ax.plot(history.history['val_accuracy'])
ax.legend(['Training Accuracy','Val Accuracy'])
bx=figure.add_subplot(122)
bx.plot(history.history['loss'])
bx.plot(history.history['val_loss'])
bx.legend(['Training Loss','Val Loss'])

In [ ]:
pred=np.argmax(model.predict(xval),axis=1)

In [ ]:
len(pred)

In [ ]:
y_test_pred = pd.Series(pred,name="Label")

submission = pd.concat([pd.Series(range(1,28001), name = "ImageId"), y_test_pred], axis = 1)

submission.to_csv("submission.csv", index = False)